**CASE 2**

In [16]:
### Library Imports ###

import pandas as pd
import numpy as np
from scipy.optimize import brentq

In [17]:
### Data Setup ###

LGD = 0.40
r = 0.03
maturities = np.array([1, 3, 5, 7, 10])
cds_rates_bps = np.array([100, 110, 120, 120, 125])
cds_rates = cds_rates_bps / 10000

**Q1**

In [18]:
##### Q1 #####

# Average Hazard Rates
average_hazard_rates_q1 = cds_rates / LGD

# Forward Hazard Rates
forward_hazard_rates_q1 = np.zeros(len(maturities))
forward_hazard_rates_q1[0] = average_hazard_rates_q1[0]
for i in range(1, len(maturities)):
    numerator = average_hazard_rates_q1[i] * maturities[i] - average_hazard_rates_q1[i-1] * maturities[i-1]
    forward_hazard_rates_q1[i] = numerator / (maturities[i] - maturities[i-1])

# Default Probabilities
survival_probs_q1 = np.exp(-average_hazard_rates_q1 * maturities)
default_probs_q1 = np.zeros(len(maturities))
default_probs_q1[0] = 1 - survival_probs_q1[0]
for i in range(1, len(maturities)):
    default_probs_q1[i] = survival_probs_q1[i-1] - survival_probs_q1[i]

results_q1 = pd.DataFrame({
    'Maturity': [f'{m}Y' for m in maturities],
    'CDS Rate (bps)': cds_rates_bps,
    '(Average) Hazard Rate': average_hazard_rates_q1,
    'Forward Hazard Rate': forward_hazard_rates_q1,
    'Default Probability': default_probs_q1
})
print("\n", results_q1.to_string(index=False))


 Maturity  CDS Rate (bps)  (Average) Hazard Rate  Forward Hazard Rate  Default Probability
      1Y             100                0.02500             0.025000             0.024690
      3Y             110                0.02750             0.028750             0.054498
      5Y             120                0.03000             0.033750             0.060103
      7Y             120                0.03000             0.030000             0.050124
     10Y             125                0.03125             0.034167             0.078969


**Q2**

In [19]:
##### Q2 #####

# Note: Case mentioned annual compounding but formula is continuous compounding, so we will use continuous compounding for discounting and survival probabilities.

def survival_probability(t, lambda_periods):
    integral = 0.0
    for t_start, t_end, lam in lambda_periods:
        if t <= t_start:
            break
        elif t <= t_end:
            integral += lam * (t - t_start)
            break
        else:
            integral += lam * (t_end - t_start)
    return np.exp(-integral)

def price_cds(maturity, cds_rate, lgd, r, lambda_periods):
    num_quarters = int(maturity * 4)
    payment_times = np.linspace(0.25, maturity, num_quarters)
    
    premium_leg = 0.0
    for i, t_i in enumerate(payment_times):
        t_prev = 0 if i == 0 else payment_times[i-1]
        dt = t_i - t_prev
        q_surv = survival_probability(t_i, lambda_periods)
        df = np.exp(-r * t_i)
        premium_leg += cds_rate * df * dt * q_surv
    
    for i, t_i in enumerate(payment_times):
        t_prev = 0 if i == 0 else payment_times[i-1]
        dt = t_i - t_prev
        t_mid = (t_i + t_prev) / 2
        q_prev = survival_probability(t_prev, lambda_periods)
        q_i = survival_probability(t_i, lambda_periods)
        df_mid = np.exp(-r * t_mid)
        premium_leg += cds_rate * df_mid * (q_prev - q_i) * (dt / 2)
    
    protection_leg = 0.0
    for i, t_i in enumerate(payment_times):
        t_prev = 0 if i == 0 else payment_times[i-1]
        t_mid = (t_i + t_prev) / 2
        q_prev = survival_probability(t_prev, lambda_periods)
        q_i = survival_probability(t_i, lambda_periods)
        df_mid = np.exp(-r * t_mid)
        protection_leg += lgd * df_mid * (q_prev - q_i)
    
    return premium_leg - protection_leg

# Strip forward hazard rates
forward_lambdas_q2 = []
lambda_periods = []

for idx in range(len(maturities)):
    mat = maturities[idx]
    def objective(lam):
        temp_periods = lambda_periods.copy()
        t_start = 0 if idx == 0 else maturities[idx-1]
        temp_periods.append((t_start, mat, lam))
        return price_cds(mat, cds_rates[idx], LGD, r, temp_periods)
    
    solution = brentq(objective, 0.0001, 1.0)
    forward_lambdas_q2.append(solution)
    t_start = 0 if idx == 0 else maturities[idx-1]
    lambda_periods.append((t_start, mat, solution))

# Calculate average hazard rates and default probabilities
average_hazard_rates_q2 = []
default_probs_q2 = []
for idx in range(len(maturities)):
    mat = maturities[idx]
    integral = sum(lam * (min(t_end, mat) - t_start) 
                   for t_start, t_end, lam in lambda_periods if t_start < mat)
    average_hazard_rates_q2.append(integral / mat)
    
    q_surv = survival_probability(mat, lambda_periods)
    q_prev = 1.0 if idx == 0 else survival_probability(maturities[idx-1], lambda_periods)
    default_probs_q2.append(q_prev - q_surv)

results_q2 = pd.DataFrame({
    'Maturity': [f'{m}Y' for m in maturities],
    'CDS Rate (bps)': cds_rates_bps,
    '(Average) Hazard Rate': average_hazard_rates_q2,
    'Forward Hazard Rate': forward_lambdas_q2,
    'Forward Default Probability': default_probs_q2
})
print("\n", results_q2.to_string(index=False))



 Maturity  CDS Rate (bps)  (Average) Hazard Rate  Forward Hazard Rate  Forward Default Probability
      1Y             100               0.024907             0.024907                     0.024599
      3Y             110               0.027472             0.028754                     0.054512
      5Y             120               0.030179             0.034239                     0.060950
      7Y             120               0.030096             0.029888                     0.049898
     10Y             125               0.031604             0.035124                     0.081013


**Q3**

In [20]:
##### Q3 #####

# Calculate 7Y CDS legs
mat_7y, cds_rate_7y = 7, cds_rates[3]
payment_times = np.linspace(0.25, mat_7y, int(mat_7y * 4))

premium_leg = sum(cds_rate_7y * np.exp(-r * t_i) * (t_i - (0 if i == 0 else payment_times[i-1])) * 
                  survival_probability(t_i, lambda_periods) 
                  for i, t_i in enumerate(payment_times))

premium_leg += sum(cds_rate_7y * np.exp(-r * (t_i + (0 if i == 0 else payment_times[i-1]))/2) * 
                   (survival_probability(0 if i == 0 else payment_times[i-1], lambda_periods) - 
                    survival_probability(t_i, lambda_periods)) * 
                   (t_i - (0 if i == 0 else payment_times[i-1])) / 2
                   for i, t_i in enumerate(payment_times))

protection_leg = sum(LGD * np.exp(-r * (t_i + (0 if i == 0 else payment_times[i-1]))/2) * 
                     (survival_probability(0 if i == 0 else payment_times[i-1], lambda_periods) - 
                      survival_probability(t_i, lambda_periods))
                     for i, t_i in enumerate(payment_times))

print(f"\nPremium Leg: {premium_leg:.10f}")
print(f"Protection Leg: {protection_leg:.10f}")
print(f"Difference: {abs(premium_leg - protection_leg):.2e}")
print(f"Result:{' Lower than tolerance' if abs(premium_leg - protection_leg) < 1e-6 else 'Failed'}")


Premium Leg: 0.0685500545
Protection Leg: 0.0685500545
Difference: 2.01e-14
Result: Lower than tolerance


**Sanity Checks**

In [21]:
### Sanity Checks ###

print("\n1. Q1 Average Hazard vs R/LGD (should match exactly):")
for i, mat in enumerate(maturities):
    expected = cds_rates[i] / LGD
    actual = average_hazard_rates_q1[i]
    print(f"   {mat}Y: {actual:.6f} vs {expected:.6f}")

print("\n2. Total Default Probability (should be < 1.0):")
print(f"   Q1: {default_probs_q1.sum():.4f}")
print(f"   Q2: {np.array(default_probs_q2).sum():.4f}")

print("\n3. Survival Probability at 10Y (should be 0 < Q < 1):")
print(f"   {survival_probability(10, lambda_periods):.4f}")

print("\n4. Q1 vs Q2 Forward Hazard Rates (bps):")
for i, mat in enumerate(maturities):
    period = f"0→{mat}Y" if i == 0 else f"{maturities[i-1]}→{mat}Y"
    q1_bps = forward_hazard_rates_q1[i] * 10000
    q2_bps = forward_lambdas_q2[i] * 10000
    print(f"   {period}: Q1={q1_bps:.2f}, Q2={q2_bps:.2f}, diff={abs(q1_bps-q2_bps):.2f}")

print("\n5. Repricing All Maturities (should be ~0):")
for i, mat in enumerate(maturities):
    pv = price_cds(mat, cds_rates[i], LGD, r, lambda_periods)
    print(f"   {mat}Y: {pv:.3e}")

print("\n6. 7Y CDS Legs from Q3:")
print(f"   Premium Leg:    {premium_leg:.6f}")
print(f"   Protection Leg: {protection_leg:.6f}")
print(f"   Difference:     {abs(premium_leg - protection_leg):.3e}")



1. Q1 Average Hazard vs R/LGD (should match exactly):
   1Y: 0.025000 vs 0.025000
   3Y: 0.027500 vs 0.027500
   5Y: 0.030000 vs 0.030000
   7Y: 0.030000 vs 0.030000
   10Y: 0.031250 vs 0.031250

2. Total Default Probability (should be < 1.0):
   Q1: 0.2684
   Q2: 0.2710

3. Survival Probability at 10Y (should be 0 < Q < 1):
   0.7290

4. Q1 vs Q2 Forward Hazard Rates (bps):
   0→1Y: Q1=250.00, Q2=249.07, diff=0.93
   1→3Y: Q1=287.50, Q2=287.54, diff=0.04
   3→5Y: Q1=337.50, Q2=342.39, diff=4.89
   5→7Y: Q1=300.00, Q2=298.88, diff=1.12
   7→10Y: Q1=341.67, Q2=351.24, diff=9.58

5. Repricing All Maturities (should be ~0):
   1Y: 1.041e-17
   3Y: -1.835e-14
   5Y: -7.458e-14
   7Y: -2.000e-14
   10Y: 4.163e-17

6. 7Y CDS Legs from Q3:
   Premium Leg:    0.068550
   Protection Leg: 0.068550
   Difference:     2.005e-14
